In [ ]:
import tempfile
from spf.dataset.fake_dataset import create_fake_dataset, fake_yaml
from spf.dataset.spf_dataset import v5spfdataset

n = 1025
noise = 0.3
nthetas = 65
orbits = 4

tmpdir = tempfile.TemporaryDirectory()
tmpdirname = tmpdir.name
temp_ds_fn = f"{tmpdirname}/sample_dataset_for_ekf_n{n}_noise{noise}"

create_fake_dataset(
    filename=temp_ds_fn, yaml_config_str=fake_yaml, n=n, noise=noise, orbits=orbits
)
ds = v5spfdataset(
    temp_ds_fn,
    nthetas=nthetas,
    ignore_qc=True,
    precompute_cache=tmpdirname,
    paired=True,
    skip_fields=set(["signal_matrix"]),
)

In [ ]:
use_real_data = False
if use_real_data:
    # ds_fn = "/mnt/md1/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_10_03_38_21_nRX2_rx_circle.zarr"
    ds_fn = "/mnt/md1/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_15_11_44_13_nRX2_bounce.zarr"
    precompute_cache_dir = "/home/mouse9911/precompute_cache_chunk16_sept"
else:
    ds_fn = temp_ds_fn
    precompute_cache_dir = tmpdirname
ds = v5spfdataset(
    ds_fn,
    nthetas=nthetas,
    ignore_qc=True,
    precompute_cache=precompute_cache_dir,
    paired=True,
    skip_fields=set(["signal_matrix", "windowed_beamformer"]),
)

In [ ]:
class SPFFilter:
    def __init__(self, ds):
        self.ds = ds

    """
    Given an idx return the known state : i.e. RX position
    """

    def our_state(self, idx):
        return None

    """
    Given current RX known state, time difference and noise level
    """

    def predict(self, our_state, dt, noise_std) -> None:
        pass

    """
    Given an idx use the internally
    """

    def update(self) -> None:
        pass

    """
    Given an idx return the observation at that point
    """

    def observation(self, idx):
        pass

    """
    Given a trajectory compute metrics over it
    """

    def metrics(self, trajectory):
        pass

    def setup(self, initial_conditions):
        pass

    def posterior_to_mu_var(self, posterior):
        return {"var": None, "mu": None}

    """
    Iterate over the dataset and generate a trajectory
    """

    def trajectory(self, initial_conditions={}, dt=1.0, noise_std=0.01):
        self.setup(initial_conditions)
        trajectory = []
        for idx in range(len(self.ds)):
            prior = self.predict(
                dt=dt,
                noise_std=noise_std,
                our_state=self.our_state(idx),
            )

            posterior = self.update(prior=prior, observation=self.observation(idx))

            trajectory.append(self.posterior_to_mu_var(posterior))

        return {"trajectory": trajectory}

In [ ]:
import os


output_prefix = "./" + os.path.basename(ds_fn) + "_"

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(10, 5))

for rx_idx in [0, 1]:
    ax[rx_idx].scatter(
        range(len(ds)),
        ds.mean_phase[f"r{rx_idx}"],
        label=f"radio{rx_idx} est phi",
        s=1.0,
        color="red",
    )
    ax[rx_idx].plot(ds.ground_truth_phis[rx_idx], label="perfect phi", color="blue")
    ax[rx_idx].plot(
        [ds[idx][rx_idx]["ground_truth_theta"] for idx in range(len(ds))],
        label=f"radio{rx_idx} gt theta",
        color="green",
    )
    ax[rx_idx].set_title(f"Radio {rx_idx}")
    ax[rx_idx].set_xlabel("Time step")
    ax[rx_idx].set_ylabel("tehta/phi")
    ax[rx_idx].legend()
    ax[rx_idx].axhline(y=0, color="r", linestyle="-")
fig.suptitle("Phase(phi) recovered from radios after segmentation")
fig.savefig(f"{output_prefix}_raw_signal.png")

In [ ]:
import numpy as np

from spf.rf import reduce_theta_to_positive_y

"""
x = [ theta dtheta/dt ]
z = [ phi ]

F = [ [ 1 dt ],
      [ 0  1 ]]

h(x) = sin(x[0]) *  2 * pi  * (d/ wavelength )

H(x) = [ dh/dx_1 , dh/dx_2 ] = [ cos(x[0]) * 2 * pi (d/ wavelength ) , 0]

"""


"""
Convert a state (x) representing [ theta dtheta/dt ] into an observation of phi
given the spacing between antennas as a fraction of wavelength
"""


def h_phi_observation_from_theta_state(
    x, antenna_spacing_in_wavelengths, radio_array_angle_offset=0
):
    assert x.ndim == 2 and x.shape[0] == 2 and x.shape[1] == 1
    return np.array(
        [
            np.sin(x[0, 0] - radio_array_angle_offset)
            * 2
            * np.pi
            * antenna_spacing_in_wavelengths
        ]
    )


"""
Compute the derivative of the observation generated from state x (theta,dtheta/dt)
with respect to state variables (theta,dtheta/dt)
"""


def hjacobian_phi_observation_from_theta_state(
    x, antenna_spacing_in_wavelengths, radio_array_angle_offset=0
):
    assert x.ndim == 2 and x.shape[0] == 2 and x.shape[1] == 1
    return np.array(
        [
            [
                np.cos(x[0, 0] - radio_array_angle_offset)
                * 2
                * np.pi
                * antenna_spacing_in_wavelengths,
                0,
            ]
        ]
    )


def single_h_phi_observation_from_theta_state(x, antenna_spacing_in_wavelengths):
    assert x[0, 0] >= -np.pi / 2 and x[0, 0] <= np.pi / 2
    return h_phi_observation_from_theta_state(x, antenna_spacing_in_wavelengths)


def single_hjacobian_phi_observation_from_theta_state(
    x, antenna_spacing_in_wavelengths
):
    assert x[0, 0] >= -np.pi / 2 and x[0, 0] <= np.pi / 2
    return hjacobian_phi_observation_from_theta_state(x, antenna_spacing_in_wavelengths)

In [ ]:
from spf.rf import reduce_theta_to_positive_y


(np.array([-np.pi * 0.7])) / np.pi

In [ ]:
from filterpy.kalman import ExtendedKalmanFilter

from spf.rf import pi_norm
from functools import cache, partial
from filterpy.common import Q_discrete_white_noise


@cache
def Q_discrete_white_noise_cached(**kwargs):
    return Q_discrete_white_noise(**kwargs)


@cache
def F_cached(dt):
    return np.eye(2) + np.array([[0, 1], [0, 0]]) * dt


def residual(a, b):
    # we are dealing in phi space here, not theta space
    # in phi space lets make sure we use the closer of the
    # two points
    return pi_norm(a - b)


class SPFKalmanFilter(ExtendedKalmanFilter, SPFFilter):
    def __init__(self, ds, rx_idx, phi_std=0.5, p=5, dynamic_R=False, **kwargs):
        super().__init__(dim_x=2, dim_z=1, **kwargs)
        self.R *= phi_std**2
        self.P *= p  # initialized as identity?

        self.ds = ds
        # flip the sign of antennas
        assert (
            ds.yaml_config["receivers"][0]["antenna-spacing-m"]
            == ds.yaml_config["receivers"][1]["antenna-spacing-m"]
        )
        antenna_spacing = -ds.yaml_config["receivers"][0]["antenna-spacing-m"]

        assert ds.wavelengths[0] == ds.wavelengths[1]
        wavelength = ds.wavelengths[0]

        self.antenna_spacing_in_wavelengths = antenna_spacing / wavelength
        self.rx_idx = rx_idx

        self.dynamic_R = dynamic_R

    def R_at_x(self):
        return 2.5 * np.exp(-((abs(pi_norm(self.x[0, 0])) - np.pi / 2) ** 2))

    def fix_x(self):
        self.x[0] = reduce_theta_to_positive_y(pi_norm(self.x[0]))
        assert self.x[0] >= -np.pi / 2 and self.x[0] <= np.pi / 2

    """
    Given current RX known state, time difference and noise level
    Predict and return prior
    """

    def predict(self, dt, noise_std):  # q_var -> noise_std
        self.F = F_cached(dt)
        self.Q = Q_discrete_white_noise_cached(
            dim=2, dt=dt, var=noise_std
        )  # TODO Cache this
        ### predict self.x
        self.x = np.dot(self.F, self.x)
        self.fix_x()
        ###

        # update covar
        self.P = np.dot(self.F, self.P).dot(self.F.T) + self.Q

    def update(self, observation):
        super().update(
            np.array(observation),
            partial(
                single_hjacobian_phi_observation_from_theta_state,
                antenna_spacing_in_wavelengths=self.antenna_spacing_in_wavelengths,
            ),
            partial(
                single_h_phi_observation_from_theta_state,
                antenna_spacing_in_wavelengths=self.antenna_spacing_in_wavelengths,
            ),
            residual=residual,
            R=self.R if not self.dynamic_R else (np.array([[self.R_at_x()]]) ** 2) * 5,
        )
        self.fix_x()

    """
    Given an idx return the observation at that point
    """

    def observation(self, idx):
        return ds[idx][self.rx_idx]["mean_phase_segmentation"]

    """
    Given a trajectory compute metrics over it
    """

    def metrics(self, trajectory):
        pass

    def setup(self, initial_conditions={}):
        self.x = np.array([[ds[rx_idx][0]["ground_truth_theta"].item()], [0]])

    def posterior_to_mu_var(self, posterior):
        return {"var": None, "mu": None}

    """
    Iterate over the dataset and generate a trajectory
    """

    def trajectory(
        self,
        initial_conditions={},
        dt=1.0,
        noise_std=0.01,
        max_iterations=None,
        debug=False,
    ):
        self.setup(initial_conditions)
        trajectory = []
        n = (
            len(self.ds)
            if max_iterations is None
            else min(max_iterations, len(self.ds))
        )
        for idx in range(n):
            # compute the prior
            self.predict(
                dt=dt,
                noise_std=noise_std,
            )

            if debug:
                hx = single_h_phi_observation_from_theta_state(
                    x=self.x,
                    antenna_spacing_in_wavelengths=self.antenna_spacing_in_wavelengths,
                )
                jacobian = single_hjacobian_phi_observation_from_theta_state(
                    x=self.x,
                    antenna_spacing_in_wavelengths=self.antenna_spacing_in_wavelengths,
                )

            # compute update = likelihood * prior
            observation = self.observation(idx)
            self.update(observation=observation)

            current_instance = {
                "mu": self.x,
                "var": self.P,
            }
            if debug:
                current_instance.update(
                    {
                        "jacobian": jacobian[0, 0],
                        "hx": hx,
                        "theta": self.x[0, 0],
                        "P_theta": self.P[0, 0],
                        "observation": observation.item(),
                    }
                )

            trajectory.append(current_instance)

        return trajectory

In [ ]:
kf = SPFKalmanFilter(
    ds=ds, rx_idx=0, phi_std=5.0, p=5
)  # , phi_std=0.5, p=5, **kwargs):
kf.trajectory(max_iterations=5)[0]

In [ ]:
import matplotlib.pyplot as plt

from spf.rf import reduce_theta_to_positive_y

fig, ax = plt.subplots(3, 2, figsize=(10, 15))

for rx_idx in [0, 1]:  # [0, 1]:
    ax[1, rx_idx].axhline(y=np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
    ax[1, rx_idx].axhline(y=-np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))

    kf = SPFKalmanFilter(
        ds=ds, rx_idx=rx_idx, phi_std=5.0, p=5, dynamic_R=True
    )  # , phi_std=0.5, p=5, **kwargs):
    trajectory = kf.trajectory(max_iterations=None, debug=True)
    jacobian = [x["jacobian"] for x in trajectory]
    zs = [x["observation"] for x in trajectory]
    # trajectory, jacobian, zs = trajectory_for_phi(rx_idx, ds)
    jacobian = np.array(jacobian)
    zs = np.array(zs)
    n = len(trajectory)
    ax[0, rx_idx].scatter(
        range(min(n, ds.mean_phase[f"r{rx_idx}"].shape[0])),
        ds.mean_phase[f"r{rx_idx}"][:n],
        label=f"r{rx_idx} estimated phi",
        s=1.0,
        alpha=1.0,
        color="red",
    )
    ax[0, rx_idx].plot(ds.ground_truth_phis[rx_idx][:n], label="perfect phi")
    ax[0, rx_idx].plot(jacobian, label="jacobian")
    ax[0, rx_idx].plot(zs, label="zs")
    ax[0, rx_idx].plot(np.clip(zs / jacobian, a_min=-5, a_max=5), label="zs/j")
    ax[1, rx_idx].plot(
        [ds[idx][rx_idx]["ground_truth_theta"] for idx in range(min(n, len(ds)))],
        label=f"r{rx_idx} gt theta",
    )
    reduced_gt_theta = np.array(
        [
            reduce_theta_to_positive_y(ds[idx][rx_idx]["ground_truth_theta"])
            for idx in range(min(n, len(ds)))
        ]
    )
    ax[1, rx_idx].plot(
        reduced_gt_theta,
        label=f"r{rx_idx} reduced gt theta",
    )

    xs = np.array([x["theta"] for x in trajectory])
    stds = np.sqrt(np.array([x["P_theta"] for x in trajectory]))
    zscores = (xs - reduced_gt_theta) / stds

    ax[1, rx_idx].plot(xs, label="EKF-x", color="orange")
    ax[1, rx_idx].fill_between(
        np.arange(xs.shape[0]),
        xs - stds,
        xs + stds,
        label="EKF-std",
        color="orange",
        alpha=0.2,
    )

    ax[0, rx_idx].set_ylabel("radio phi")

    ax[0, rx_idx].legend()
    ax[0, rx_idx].set_title(f"Radio {rx_idx}")
    ax[1, rx_idx].legend()
    ax[1, rx_idx].set_xlabel("time step")
    ax[1, rx_idx].set_ylabel("radio theta")

    ax[2, rx_idx].hist(zscores.reshape(-1), bins=25)
fig.suptitle("Single ladies (radios) EKF")
fig.savefig(f"{output_prefix}_single_ladies_ekf.png")

In [ ]:
import numpy as np


def paired_h_phi_observation_from_theta_state(
    x, antenna_spacing_in_wavelengths, radio_array_angle_offsets
):
    return np.vstack(
        [
            h_phi_observation_from_theta_state(
                x,
                antenna_spacing_in_wavelengths=antenna_spacing_in_wavelengths,
                radio_array_angle_offset=radio_array_angle_offsets[0],
            ),
            h_phi_observation_from_theta_state(
                x,
                antenna_spacing_in_wavelengths=antenna_spacing_in_wavelengths,
                radio_array_angle_offset=radio_array_angle_offsets[1],
            ),
        ]
    )


def paired_hjacobian_phi_observation_from_theta_state(
    x, antenna_spacing_in_wavelengths, radio_array_angle_offsets
):
    return np.vstack(
        [
            hjacobian_phi_observation_from_theta_state(
                x,
                antenna_spacing_in_wavelengths=antenna_spacing_in_wavelengths,
                radio_array_angle_offset=radio_array_angle_offsets[0],
            ),
            hjacobian_phi_observation_from_theta_state(
                x,
                antenna_spacing_in_wavelengths=antenna_spacing_in_wavelengths,
                radio_array_angle_offset=radio_array_angle_offsets[1],
            ),
        ]
    )

In [ ]:
class SPFPairedKalmanFilter(ExtendedKalmanFilter, SPFFilter):
    def __init__(self, ds, phi_std=0.5, p=5, dynamic_R=False, **kwargs):
        super().__init__(dim_x=2, dim_z=2, **kwargs)
        self.R *= phi_std**2
        self.P *= p  # initialized as identity?

        self.ds = ds
        # flip the sign of antennas
        assert (
            ds.yaml_config["receivers"][0]["antenna-spacing-m"]
            == ds.yaml_config["receivers"][1]["antenna-spacing-m"]
        )
        antenna_spacing = -ds.yaml_config["receivers"][0]["antenna-spacing-m"]

        assert ds.wavelengths[0] == ds.wavelengths[1]
        wavelength = ds.wavelengths[0]

        self.radio_array_angle_offsets = [
            ds.yaml_config["receivers"][0]["theta-in-pis"] * np.pi,
            ds.yaml_config["receivers"][1]["theta-in-pis"] * np.pi,
        ]

        self.antenna_spacing_in_wavelengths = antenna_spacing / wavelength

        self.dynamic_R = dynamic_R

    def R_at_x(self, angle):
        return 2.5 * np.exp(-((abs(pi_norm(angle)) - np.pi / 2) ** 2))

    def fix_x(self):
        self.x[0] = pi_norm(self.x[0])

    """
    Given current RX known state, time difference and noise level
    Predict and return prior
    """

    def predict(self, dt, noise_std):  # q_var -> noise_std
        self.F = F_cached(dt)
        self.Q = Q_discrete_white_noise_cached(
            dim=2, dt=dt, var=noise_std
        )  # TODO Cache this
        ### predict self.x
        self.x = np.dot(self.F, self.x)
        self.fix_x()
        ###

        # update covar
        self.P = np.dot(self.F, self.P).dot(self.F.T) + self.Q

    def update(self, observation):
        r = np.array(
            [
                [
                    self.R_at_x(
                        pi_norm(self.x[0, 0] - self.radio_array_angle_offsets[0])
                    ),
                    0,
                ],
                [
                    0,
                    self.R_at_x(
                        pi_norm(self.x[0, 0] - self.radio_array_angle_offsets[1])
                    ),
                ],
            ]
        )
        super().update(
            np.array(observation),
            partial(
                paired_hjacobian_phi_observation_from_theta_state,
                antenna_spacing_in_wavelengths=self.antenna_spacing_in_wavelengths,
                radio_array_angle_offsets=self.radio_array_angle_offsets,
            ),
            partial(
                paired_h_phi_observation_from_theta_state,
                antenna_spacing_in_wavelengths=self.antenna_spacing_in_wavelengths,
                radio_array_angle_offsets=self.radio_array_angle_offsets,
            ),
            residual=residual,
            R=self.R if not self.dynamic_R else r,
        )
        self.fix_x()

    """
    Given an idx return the observation at that point
    """

    def observation(self, idx):
        return np.vstack(
            [
                ds[idx][0]["mean_phase_segmentation"],
                ds[idx][1]["mean_phase_segmentation"],
            ]
        )

    """
    Given a trajectory compute metrics over it
    """

    def metrics(self, trajectory):
        pass

    def setup(self, initial_conditions={}):
        self.x = np.array([[ds[rx_idx][0]["craft_ground_truth_theta"].item()], [0]])

    def posterior_to_mu_var(self, posterior):
        return {"var": None, "mu": None}

    """
    Iterate over the dataset and generate a trajectory
    """

    def trajectory(
        self,
        initial_conditions={},
        dt=1.0,
        noise_std=0.01,
        max_iterations=None,
        debug=False,
    ):
        self.setup(initial_conditions)
        trajectory = []
        n = (
            len(self.ds)
            if max_iterations is None
            else min(max_iterations, len(self.ds))
        )
        for idx in range(n):
            # compute the prior
            self.predict(
                dt=dt,
                noise_std=noise_std,
            )

            if debug:
                hx = paired_h_phi_observation_from_theta_state(
                    x=self.x,
                    antenna_spacing_in_wavelengths=self.antenna_spacing_in_wavelengths,
                    radio_array_angle_offsets=self.radio_array_angle_offsets,
                )
                jacobian = paired_hjacobian_phi_observation_from_theta_state(
                    x=self.x,
                    antenna_spacing_in_wavelengths=self.antenna_spacing_in_wavelengths,
                    radio_array_angle_offsets=self.radio_array_angle_offsets,
                )

            # compute update = likelihood * prior
            observation = self.observation(idx)
            self.update(observation=observation)

            current_instance = {
                "mu": self.x,
                "var": self.P,
            }
            if debug:
                current_instance.update(
                    {
                        "jacobian": jacobian[0, 0],
                        "hx": hx,
                        "theta": self.x[0, 0],
                        "P_theta": self.P[0, 0],
                        "observation": observation,
                    }
                )

            trajectory.append(current_instance)

        return trajectory

In [ ]:
kf = SPFPairedKalmanFilter(
    ds=ds, phi_std=5.0, p=5, dynamic_R=False
)  # , phi_std=0.5, p=5, **kwargs):
trajectory = kf.trajectory(max_iterations=5, debug=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from spf.rf import reduce_theta_to_positive_y

fig, ax = plt.subplots(3, 1, figsize=(10, 15))

ax[1].axhline(y=np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
ax[1].axhline(y=-np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
kf = SPFPairedKalmanFilter(ds=ds, phi_std=5.0, p=5, dynamic_R=False)
trajectory = kf.trajectory(max_iterations=None, debug=True)

n = len(trajectory)
ax[0].scatter(
    range(min(n, ds.mean_phase[f"r{rx_idx}"].shape[0])),
    ds.mean_phase[f"r{rx_idx}"][:n],
    label=f"r{rx_idx} estimated phi",
    s=1.0,
    alpha=1.0,
    color="red",
)
ax[0].plot(ds.ground_truth_phis[rx_idx][:n], label="perfect phi")
ground_truth_theta = [
    pi_norm(ds[idx][0]["craft_y_rad"].item()) for idx in range(len(trajectory))
]
ax[1].plot(
    ground_truth_theta,
    label="craft gt theta",
)

xs = np.array([x["theta"] for x in trajectory])
stds = np.sqrt(np.array([x["P_theta"] for x in trajectory]))
zscores = (xs - np.array(ground_truth_theta)) / stds

ax[1].plot(xs, label="EKF-x", color="orange")
ax[1].fill_between(
    np.arange(xs.shape[0]),
    xs - stds,
    xs + stds,
    label="EKF-std",
    color="orange",
    alpha=0.2,
)

ax[0].set_ylabel("radio phi")

ax[0].legend()
ax[0].set_title(f"Radio")
ax[1].legend()
ax[1].set_xlabel("time step")
ax[1].set_ylabel("radio theta")

ax[2].hist(zscores.reshape(-1), bins=25)
# fig.suptitle("Single ladies (radios) EKF")
# fig.savefig(f"{output_prefix}_single_ladies_ekf.png")

In [ ]:
def pairedXY_h_phi_observation_from_theta_state(
    x, antenna_spacing_in_wavelengths, radio_array_angle_offsets
):
    rel_x, rel_y = x[0, 0] - x[4, 0], x[1, 0] - x[5, 0]
    target_theta = pi_norm(np.arctan2(rel_x, rel_y))
    two_state_theta_dtheta = np.array([[target_theta], [0]])
    return np.vstack(
        [
            h_phi_observation_from_theta_state(
                two_state_theta_dtheta,
                antenna_spacing_in_wavelengths=antenna_spacing_in_wavelengths,
                radio_array_angle_offset=radio_array_angle_offsets[0],
            ),
            h_phi_observation_from_theta_state(
                two_state_theta_dtheta,
                antenna_spacing_in_wavelengths=antenna_spacing_in_wavelengths,
                radio_array_angle_offset=radio_array_angle_offsets[1],
            ),
        ]
    )


def pairedXY_hjacobian_phi_observation_from_theta_state(
    x, antenna_spacing_in_wavelengths, radio_array_angle_offsets
):
    rel_x, rel_y = x[0, 0] - x[4, 0], x[1, 0] - x[5, 0]
    target_theta = pi_norm(np.arctan2(rel_x, rel_y))
    # print("Target", target_theta)
    two_state_theta_dtheta = np.array([[target_theta], [0]])
    two_state_jacobian = np.vstack(
        [
            hjacobian_phi_observation_from_theta_state(
                two_state_theta_dtheta,
                antenna_spacing_in_wavelengths=antenna_spacing_in_wavelengths,
                radio_array_angle_offset=radio_array_angle_offsets[0],
            ),
            hjacobian_phi_observation_from_theta_state(
                two_state_theta_dtheta,
                antenna_spacing_in_wavelengths=antenna_spacing_in_wavelengths,
                radio_array_angle_offset=radio_array_angle_offsets[1],
            ),
        ]
    )
    d2 = rel_x**2 + rel_y**2
    dtheta_drel_x = rel_y / d2
    dtheta_drel_y = -rel_x / d2
    dphi0_dtheta = two_state_jacobian[0, 0]
    dphi1_dtheta = two_state_jacobian[1, 0]
    jacobian = np.zeros((2, 6))
    jacobian[0, 0] = dphi0_dtheta * dtheta_drel_x  #  * drel_x/dx
    jacobian[0, 1] = dphi0_dtheta * dtheta_drel_y  # * drel_y/dy
    jacobian[1, 0] = dphi1_dtheta * dtheta_drel_x  # * drel_x/dx
    jacobian[1, 1] = dphi1_dtheta * dtheta_drel_y  # * drel_y/dy
    # print("jacobian",jacobian)
    return jacobian

In [ ]:
np.arctan2(-0.01, 2)

In [ ]:
Q_discrete_white_noise(dim=2, dt=1, block_size=2, order_by_dim=False)

In [ ]:
Q_discrete_white_noise

In [ ]:
@cache
def Qxy_discrete_white_noise_cached(**kwargs):
    q = np.zeros((6, 6))
    q[:4, :4] = Q_discrete_white_noise(**kwargs)
    return q


@cache
def Fxy_cached(dt):
    return (
        np.eye(6)
        + np.array(
            [
                [0, 0, 1, 0, 0, 0],
                [0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, 0, 0],  # no updates for dtx_x/dt
                [0, 0, 0, 0, 0, 0],  # no updates for dtx_y/dt
                [0, 0, 0, 0, 0, 0],  # no updates for rx_x
                [0, 0, 0, 0, 0, 0],  # no updates for rx_y
            ]
        )
        * dt
    )


class SPFPairedXYKalmanFilter(ExtendedKalmanFilter, SPFFilter):
    def __init__(self, ds, phi_std=0.5, p=5, dynamic_R=False, **kwargs):
        # state x = [ tx_x, tx_y, dtx_x/dt, dtx_y/dt, rx_x, rx_y] , x,y relative to (0,0) not craft
        super().__init__(dim_x=6, dim_z=2, **kwargs)
        self.R *= phi_std**2
        self.P *= p  # initialized as identity?

        self.ds = ds
        # flip the sign of antennas
        assert (
            ds.yaml_config["receivers"][0]["antenna-spacing-m"]
            == ds.yaml_config["receivers"][1]["antenna-spacing-m"]
        )
        antenna_spacing = -ds.yaml_config["receivers"][0]["antenna-spacing-m"]

        assert ds.wavelengths[0] == ds.wavelengths[1]
        wavelength = ds.wavelengths[0]

        self.radio_array_angle_offsets = [
            ds.yaml_config["receivers"][0]["theta-in-pis"] * np.pi,
            ds.yaml_config["receivers"][1]["theta-in-pis"] * np.pi,
        ]

        self.antenna_spacing_in_wavelengths = antenna_spacing / wavelength

        self.dynamic_R = dynamic_R

    def R_at_x(self, angle):
        return 2.5 * np.exp(-((abs(pi_norm(angle)) - np.pi / 2) ** 2))

    def fix_x(self):
        # self.x[[2, 3]] = np.clip(self.x[[2, 3]], a_min=-0.1, a_max=0.1)
        pass

    """
    Given current RX known state, time difference and noise level
    Predict and return prior
    """

    def predict(self, dt, noise_std):  # q_var -> noise_std
        self.F = Fxy_cached(dt)
        self.Q = Qxy_discrete_white_noise_cached(
            dim=2, dt=dt, var=noise_std, block_size=2, order_by_dim=False
        )  # TODO Cache thisQ_discrete_white_noise(dim=2, dt=1,block_size=2,order_by_dim=False)
        ### predict self.x
        self.x = np.dot(self.F, self.x)
        self.fix_x()
        ###
        # print("P", self.P.shape, "Q", self.Q.shape)

        # update covar
        self.P = np.dot(self.F, self.P).dot(self.F.T) + self.Q

    def update(self, observation):

        rel_x, rel_y = self.x[0, 0] - self.x[4, 0], self.x[1, 0] - self.x[5, 0]
        target_theta = np.arctan2(rel_x, rel_y)
        r = np.array(
            [
                [
                    self.R_at_x(
                        pi_norm(target_theta - self.radio_array_angle_offsets[0])
                    ),
                    0,
                ],
                [
                    0,
                    self.R_at_x(
                        pi_norm(target_theta - self.radio_array_angle_offsets[1])
                    ),
                ],
            ]
        )
        # print("Update", self.x)
        super().update(
            np.array(observation),
            partial(
                pairedXY_hjacobian_phi_observation_from_theta_state,
                antenna_spacing_in_wavelengths=self.antenna_spacing_in_wavelengths,
                radio_array_angle_offsets=self.radio_array_angle_offsets,
            ),
            partial(
                pairedXY_h_phi_observation_from_theta_state,
                antenna_spacing_in_wavelengths=self.antenna_spacing_in_wavelengths,
                radio_array_angle_offsets=self.radio_array_angle_offsets,
            ),
            residual=residual,
            R=self.R if not self.dynamic_R else r,
        )
        self.fix_x()

    """
    Given an idx return the observation at that point
    """

    def observation(self, idx):
        return np.vstack(
            [
                ds[idx][0]["mean_phase_segmentation"],
                ds[idx][1]["mean_phase_segmentation"],
            ]
        )

    """
    Given a trajectory compute metrics over it
    """

    def metrics(self, trajectory):
        pass

    def setup(self, initial_conditions={}):
        self.x = np.array(
            [
                [ds[0][0]["tx_pos_x_mm"].item()],
                [ds[0][0]["tx_pos_y_mm"].item()],
                [0],
                [0],
                [ds[0][0]["rx_pos_x_mm"].item()],
                [ds[0][0]["rx_pos_y_mm"].item()],
            ]
        )

    def posterior_to_mu_var(self, posterior):
        return {"var": None, "mu": None}

    """
    Iterate over the dataset and generate a trajectory
    """

    def trajectory(
        self,
        initial_conditions={},
        dt=1.0,
        noise_std=0.01,
        max_iterations=None,
        debug=False,
    ):
        self.setup(initial_conditions)
        trajectory = []
        n = (
            len(self.ds)
            if max_iterations is None
            else min(max_iterations, len(self.ds))
        )
        for idx in range(n):
            self.P[4:, 4:] = 0
            self.predict(
                dt=dt,
                noise_std=noise_std,
            )

            observation = self.observation(idx)

            self.x[4] = ds[idx][0]["rx_pos_x_mm"].item()
            self.x[5] = ds[idx][0]["rx_pos_y_mm"].item()
            if debug:
                hx = pairedXY_h_phi_observation_from_theta_state(
                    x=self.x,
                    antenna_spacing_in_wavelengths=self.antenna_spacing_in_wavelengths,
                    radio_array_angle_offsets=self.radio_array_angle_offsets,
                )
                jacobian = pairedXY_hjacobian_phi_observation_from_theta_state(
                    x=self.x,
                    antenna_spacing_in_wavelengths=self.antenna_spacing_in_wavelengths,
                    radio_array_angle_offsets=self.radio_array_angle_offsets,
                )

            self.update(observation=observation)

            current_instance = {
                "mu": self.x,
                "var": self.P[:4, :4],
            }
            if debug:
                self.x[4] = ds[idx][0]["rx_pos_x_mm"].item()
                self.x[5] = ds[idx][0]["rx_pos_y_mm"].item()
                rel_x, rel_y = self.x[0, 0] - self.x[4, 0], self.x[1, 0] - self.x[5, 0]
                target_theta = pi_norm(np.arctan2(rel_x, rel_y))
                current_instance.update(
                    {
                        "jacobian": jacobian[0, 0],
                        "hx": hx,
                        "theta": target_theta,
                        "P_theta": 0.01,  # self.P[0, 0],
                        "observation": observation,
                    }
                )

            trajectory.append(current_instance)

        return trajectory

In [ ]:
kf = SPFPairedXYKalmanFilter(
    ds=ds, phi_std=5.0, p=0.001, dynamic_R=False
)  # , phi_std=0.5, p=5, **kwargs):
trajectory = kf.trajectory(max_iterations=5, debug=True, dt=0.05)
# trajectory

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from spf.rf import reduce_theta_to_positive_y

fig, ax = plt.subplots(3, 1, figsize=(10, 15))

ax[1].axhline(y=np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
ax[1].axhline(y=-np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
kf = SPFPairedXYKalmanFilter(ds=ds, phi_std=5, p=0.1, dynamic_R=True)
trajectory = kf.trajectory(
    max_iterations=1600, debug=True, noise_std=10.0, dt=1
)  # , noise_std=1)

n = len(trajectory)
ax[0].scatter(
    range(min(n, ds.mean_phase[f"r{rx_idx}"].shape[0])),
    ds.mean_phase[f"r{rx_idx}"][:n],
    label=f"r{rx_idx} estimated phi",
    s=1.0,
    alpha=1.0,
    color="red",
)
ax[0].plot(ds.ground_truth_phis[rx_idx][:n], label="perfect phi")
ground_truth_theta = [
    pi_norm(ds[idx][0]["craft_y_rad"].item()) for idx in range(len(trajectory))
]
ax[1].plot(
    ground_truth_theta,
    label="craft gt theta",
)

xs = np.array([x["theta"] for x in trajectory])
stds = np.sqrt(np.array([x["P_theta"] for x in trajectory]))
zscores = (xs - np.array(ground_truth_theta)) / stds

ax[1].plot(xs, label="EKF-x", color="orange")
ax[1].fill_between(
    np.arange(xs.shape[0]),
    xs - stds,
    xs + stds,
    label="EKF-std",
    color="orange",
    alpha=0.2,
)

ax[0].set_ylabel("radio phi")

ax[0].legend()
ax[0].set_title(f"Radio")
ax[1].legend()
ax[1].set_xlabel("time step")
ax[1].set_ylabel("radio theta")

# ax[2].hist(zscores.reshape(-1), bins=25)
# fig.suptitle("Single ladies (radios) EKF")
# fig.savefig(f"{output_prefix}_single_ladies_ekf.png")

pos_x = np.array([x["mu"][0] for x in trajectory])
pos_y = np.array([x["mu"][1] for x in trajectory])
rpos_x = np.array([x["mu"][4] for x in trajectory])
rpos_y = np.array([x["mu"][5] for x in trajectory])
vel_x = np.array([x["mu"][2] for x in trajectory])
vel_y = np.array([x["mu"][3] for x in trajectory])
gt_x = [ds[idx][0]["tx_pos_x_mm"].item() for idx in range(len(trajectory))]
gt_y = [ds[idx][0]["tx_pos_y_mm"].item() for idx in range(len(trajectory))]
ax[2].plot(gt_x, color="red")
ax[2].plot(gt_y, color="green")
ax[2].scatter(range(len(pos_x)), pos_x, color="red")
ax[2].scatter(range(len(pos_y)), pos_y, color="green")
ax[2].plot(range(len(rpos_x)), rpos_x, color="blue")
ax[2].plot(range(len(rpos_y)), rpos_y, color="black")
# ax[2].scatter(range(len(vel_x)), vel_x, color="red")
# ax[2].scatter(range(len(vel_y)), vel_y, color="green")

In [ ]:
ds[3][0]["tx_pos_x_mm"]

In [ ]:
import torch

tx_pos = torch.vstack(
    [
        torch.tensor([ds[idx][0]["tx_pos_x_mm"], ds[idx][0]["tx_pos_y_mm"]])
        for idx in range(10)
    ]
)
rx_pos = torch.vstack(
    [
        torch.tensor([ds[idx][0]["rx_pos_x_mm"], ds[idx][0]["rx_pos_y_mm"]])
        for idx in range(10)
    ]
)
craft_theta = [
    torch.tensor(ds[idx][0]["craft_ground_truth_theta"]) for idx in range(10)
]
d = tx_pos - rx_pos
torch.arctan2(d[:, 0], d[:, 1]) - torch.tensor(craft_theta)

In [ ]:
#craft_ground_truth_theta
#tx_pos_x_mm,tx_pos_y_mm
#rx_pos_x_mm,rx_pos_y_mm
# plot g.t. theta vs compute

tx_pos = torch.vstack(
                [
                    self.cached_keys[ridx]["tx_pos_x_mm"],
                    self.cached_keys[ridx]["tx_pos_y_mm"],
                ]
            )
            rx_pos = torch.vstack(
                [
                    self.cached_keys[ridx]["rx_pos_x_mm"],
                    self.cached_keys[ridx]["rx_pos_y_mm"],
                ]
            )

            # compute the angle of the tx with respect to rx
            d = tx_pos - rx_pos

            rx_to_tx_theta = torch.arctan2(d[0], d[1])
#ds[0]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from spf.rf import reduce_theta_to_positive_y

fig, ax = plt.subplots(3, 1, figsize=(10, 15))

ax[1].axhline(y=np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
ax[1].axhline(y=-np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
trajectory = trajectory_for_phi_paired(ds)
n = len(trajectory)
ax[0].scatter(
    range(min(n, ds.mean_phase[f"r{rx_idx}"].shape[0])),
    ds.mean_phase[f"r{rx_idx}"][:n],
    label=f"r{rx_idx} estimated phi",
    s=1.0,
    alpha=1.0,
    color="red",
)
ax[0].plot(ds.ground_truth_phis[rx_idx][:n], label="perfect phi")
# ax[1].plot(
#     [ds[idx][rx_idx]["ground_truth_theta"] for idx in range(min(n, len(ds)))],
#     label=f"r{rx_idx} gt theta",
# )
ax[1].plot(
    [torch_pi_norm(ds[idx][0]["craft_y_rad"].item()) for idx in range(len(ds))],
    label="craft gt theta",
)
reduced_gt_theta = np.array(
    [
        reduce_theta_to_positive_y(ds[idx][rx_idx]["ground_truth_theta"])
        for idx in range(min(n, len(ds)))
    ]
)
ax[1].plot(
    reduced_gt_theta,
    label=f"r{rx_idx} reduced gt theta",
)

xs = np.array([x["theta"] for x in trajectory])
stds = np.sqrt(np.array([x["P_theta"] for x in trajectory]))
zscores = (xs - reduced_gt_theta) / stds

ax[1].plot(xs, label="EKF-x", color="orange")
ax[1].fill_between(
    np.arange(xs.shape[0]),
    xs - stds,
    xs + stds,
    label="EKF-std",
    color="orange",
    alpha=0.2,
)

ax[0].set_ylabel("radio phi")

ax[0].legend()
ax[0].set_title(f"Radio {rx_idx}")
ax[1].legend()
ax[1].set_xlabel("time step")
ax[1].set_ylabel("radio theta")

ax[2].hist(zscores.reshape(-1), bins=25)
# fig.suptitle("Single ladies (radios) EKF")
# fig.savefig(f"{output_prefix}_single_ladies_ekf.png")

In [ ]:
Q_discrete_white_noise(dim=2, dt=0.05, var=1.0)

In [ ]:
Q_discrete_white_noise(dim=2, dt=1.0, var=0.0001)

In [ ]:
def fx(x, dt):
    F = F_cached(dt)
    return np.dot(F, x)


def hx_paired_ukf(x):
    return np.array(
        [
            np.sin(x[0] - offsets[0]) * antenna_spacing * 2 * np.pi / wavelength,
            np.sin(x[0] - offsets[1]) * antenna_spacing * 2 * np.pi / wavelength,
        ]
    )


def ukf_paired_residual(a, b):
    # return pi_norm(a - b)
    return np.array([residual(a[0], b[0]), residual(a[1], b[1])])


def ukf_paired_residual_x(a, b):
    # return pi_norm(a - b)
    return np.array([pi_norm(a[0] - b[0]), a[0] - b[0]])


def state_mean(sigmas, Wm):
    x = np.zeros(2)
    sum_sin, sum_cos = 0.0, 0.0

    for i in range(len(sigmas)):
        s = sigmas[i]
        x[1] += s[1] * Wm[i]
        sum_sin += np.sin(s[0]) * Wm[i]
        sum_cos += np.cos(s[0]) * Wm[i]
    x[1] /= len(sigmas)
    x[0] = np.arctan2(sum_sin, sum_cos)
    return x

In [ ]:
from filterpy.kalman.sigma_points import MerweScaledSigmaPoints
from filterpy.kalman import UnscentedKalmanFilter

dt = 1.0
points = MerweScaledSigmaPoints(2, alpha=0.1, beta=2.0, kappa=0)  # -1)
kf = UnscentedKalmanFilter(
    dim_x=2,
    dim_z=2,
    dt=dt,
    fx=fx,
    hx=hx_paired_ukf,
    points=points,
    residual_z=ukf_paired_residual,
    # x_mean_fn=state_mean,
    residual_x=ukf_paired_residual_x,
)
kf.x = np.array([0, 0])  # initial state
kf.P[0, 0] *= 5  # initial uncertainty
kf.P[1, 1] *= 0.2  # initial uncertainty
z_std = 5
kf.R = np.diag([z_std**2, z_std**2])  # 1 standard
kf.Q = Q_discrete_white_noise(dim=2, dt=dt, var=0.01**2)

trajectory = []

#    for idx in range(min(2500, len(ds))):
for idx in range(min(800, len(ds))):
    print(kf.x)
    kf.predict()
    kf.x[0] = pi_norm(kf.x[0])
    # kf.x[0] = pi_norm(kf.x[0])
    print(kf.x)
    z = np.array(
        [
            ds[idx][0]["mean_phase_segmentation"],
            ds[idx][1]["mean_phase_segmentation"],
        ]
    )
    # print(kf.x)
    kf.update(z)
    kf.x[0] = pi_norm(kf.x[0])
    trajectory.append({"theta": kf.x[0]})  # , "P_theta": ekf.P[0, 0]})
    # print(kf.x, "log-likelihood", kf.log_likelihood)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from spf.rf import reduce_theta_to_positive_y

fig, ax = plt.subplots(3, 1, figsize=(10, 15))

ax[1].axhline(y=np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
ax[1].axhline(y=-np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
# trajectory = trajectory_for_phi_paired(ds)
n = len(trajectory)
ax[0].scatter(
    range(min(n, ds.mean_phase[f"r{rx_idx}"].shape[0])),
    ds.mean_phase[f"r{rx_idx}"][:n],
    label=f"r{rx_idx} estimated phi",
    s=1.0,
    alpha=1.0,
    color="red",
)
ax[0].plot(ds.ground_truth_phis[rx_idx][:n], label="perfect phi")
# ax[1].plot(
#     [ds[idx][rx_idx]["ground_truth_theta"] for idx in range(min(n, len(ds)))],
#     label=f"r{rx_idx} gt theta",
# )
ax[1].plot(
    [torch_pi_norm(ds[idx][0]["craft_y_rad"].item()) for idx in range(len(ds))],
    label="craft gt theta",
)
reduced_gt_theta = np.array(
    [
        reduce_theta_to_positive_y(ds[idx][rx_idx]["ground_truth_theta"])
        for idx in range(min(n, len(ds)))
    ]
)
ax[1].plot(
    reduced_gt_theta,
    label=f"r{rx_idx} reduced gt theta",
)

xs = np.array([x["theta"] for x in trajectory])
# stds = np.sqrt(np.array([x["P_theta"] for x in trajectory]))
# zscores = (xs - reduced_gt_theta) / stds

ax[1].plot(xs, label="EKF-x", color="orange")
# ax[1].fill_between(
#     np.arange(xs.shape[0]),
#     xs - stds,
#     xs + stds,
#     label="EKF-std",
#     color="orange",
#     alpha=0.2,
# )

ax[0].set_ylabel("radio phi")

ax[0].legend()
ax[0].set_title(f"Radio {rx_idx}")
ax[1].legend()
ax[1].set_xlabel("time step")
ax[1].set_ylabel("radio theta")

ax[2].hist(zscores.reshape(-1), bins=25)
# fig.suptitle("Single ladies (radios) EKF")
# fig.savefig(f"{output_prefix}_single_ladies_ekf.png")

In [ ]:
# lets try a particle filter!

In [ ]:
!pip install pypfilt

In [ ]:
class GaussianWalk(pypfilt.Model):
    def field_types(self, ctx):
        return [("x", np.dtype(float))]

    def update(self, ctx, time_step, is_fs, prev, curr):
        """Perform a single time-step."""
        rnd = ctx.component["random"]["model"]
        step = rnd.normal(loc=0, scale=1, size=curr.shape)
        curr["x"] = prev["x"] + step

In [ ]:
class Lorenz63(OdeModel):
    def field_types(self, ctx):
        r"""
        Define the state vector :math:`[\sigma, \rho, \beta, x, y, z]^T`.
        """
        return [
            ("tehta", float),
            ("tehta_vel", float),
        ]

    def d_dt(self, time, xt, ctx, is_forecast):
        rates = np.zeros(xt.shape, xt.dtype)
        rates["theta"] = xt["tehta_vel"]
        return rates

In [ ]:
np

In [ ]:
from spf.dataset.spf_dataset import v5spfdataset
import numpy as np

from spf.rf import pi_norm

ds_fn = "/mnt/md0/spf/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_10_03_38_21_nRX2_rx_circle.zarr"
# ds_fn = "/mnt/md0/spf/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_15_11_44_13_nRX2_bounce.zarr"


nthetas = 65
ds = v5spfdataset(
    ds_fn,
    nthetas=nthetas,
    ignore_qc=True,
    precompute_cache="/home/mouse9911/precompute_cache_chunk16",
    paired=True,
    skip_fields=set(["signal_matrix"]),
)

In [ ]:
import pickle

full_p = pickle.load(open("full_p.pkl", "rb"))["full_p"]

In [ ]:
np.random.randn(4, 3)

In [ ]:
import numpy as np
import scipy

from filterpy.monte_carlo import systematic_resample

# flip the order of the antennas
antenna_spacing = -ds.yaml_config["receivers"][0]["antenna-spacing-m"]
assert antenna_spacing == -ds.yaml_config["receivers"][1]["antenna-spacing-m"]

wavelength = ds.wavelengths[0]
assert wavelength == ds.wavelengths[1]


def resample_from_index(particles, weights, indexes):
    # print(indexes)
    particles[:] = particles[indexes]
    # weights[:] = weights[indexes]
    # add noise to the new samples
    noise = np.random.randn(*particles.shape)
    # noise[:, 0] *= 0.01
    # noise[:, 1] *= 0.01
    noise[:, 0] *= 0.01
    noise[:, 1] *= 0.001
    change_mask = indexes[:-1] == indexes[1:]
    particles[1:][change_mask] += noise[1:][change_mask]
    # weights.resize(len(particles))
    weights.fill(1.0 / len(weights))
    weights[1:][change_mask] *= 0.01
    weights /= sum(weights)  # normalize


def create_gaussian_particles(mean, std, N):
    particles = np.empty((N, 2))
    particles[:, 0] = mean[0] + (np.random.randn(N) * std[0])
    particles[:, 1] = mean[1] + (np.random.randn(N) * std[1])
    return particles


def predict(particles, std, dt=1.0):
    N = len(particles)
    particles[:, 0] += dt * particles[:, 1]
    # particles[:, 0] += np.random.randn(particles.shape[0]) * 0.0001
    # particles[:, 1] += np.random.randn(particles.shape[0]) * 0.0001


def fix_particles(particles):
    while np.abs(particles[:, 0]).max() > np.pi / 2:
        mask = np.abs(particles[:, 0]) > np.pi / 2
        particles[mask, 0] = np.sign(particles[mask, 0]) * np.pi - particles[mask, 0]
        particles[mask, 1] *= -1
    return particles


def hx_pf(x):
    return np.sin(x[:, 0]) * antenna_spacing * 2 * np.pi / wavelength


def theta_phi_to_p(theta, phi, full_p):
    theta_bins = full_p.shape[0]
    phi_bins = full_p.shape[1]
    theta_bin = int(theta_bins * (theta + np.pi) / (2 * np.pi)) % theta_bins
    phi_bin = int(phi_bins * (phi + np.pi) / (2 * np.pi)) % phi_bins
    return full_p[theta_bin, phi_bin]


def update(particles, weights, z, R):
    # weights *= scipy.stats.norm(hx_pf(particles), R).pdf(z)
    # print(weights.shape, z.shape, particles.shape)
    # print(z)
    for idx in range(weights.shape[0]):
        weights[idx] *= theta_phi_to_p(particles[idx, 0], z, full_p=full_p)
        # weights[idx] *= scipy.stats.norm(0, 0.0001).pdf(particles[idx, 1])
    weights += 1.0e-300  # avoid round-off to zero
    weights /= sum(weights)  # normalize


def estimate(particles, weights):
    mean = np.average(particles, weights=weights, axis=0)
    var = np.average((particles - mean) ** 2, weights=weights, axis=0)
    return mean, var


def neff(weights):
    return 1.0 / np.sum(np.square(weights))


def pf_trajectory_for_phi(rx_idx, ds):
    N = 128 * 8 * 2
    particles = create_gaussian_particles(mean=np.array([0, 0]), std=(2, 0.01), N=N)
    weights = np.ones((N,)) / N
    trajectory = []
    thetas = []
    vs = []
    for idx in range(len(ds)):
        particles = fix_particles(particles)
        predict(particles=particles, std=(0.5, 0.1), dt=0.1)
        particles = fix_particles(particles)
        z = np.array(ds[idx][rx_idx]["mean_phase_segmentation"])
        update(particles=particles, weights=weights, z=z, R=2)
        particles = fix_particles(particles)
        # print(neff(weights))
        # print(particles.shape)
        # resample if too few effective particles
        if neff(weights) < N / 2:
            # print("RESAMPLE")
            indexes = systematic_resample(weights)
            resample_from_index(particles, weights, indexes)
            # assert np.allclose(weights, 1 / N)
        mu, var = estimate(particles, weights)
        # particles = create_gaussian_particles(mean=mu, std=np.sqrt(var), N=N)
        trajectory.append({"theta": mu[0]})
        for particle in particles:
            thetas.append((idx, particle[0]))
            vs.append((idx, particle[1]))
    return trajectory

In [ ]:
# plt.imshow(
#     np.array(
#         [
#             theta_phi_to_p(np.pi / 4, phi, full_p)
#             for phi in np.linspace(-np.pi, np.pi, 50)
#         ]
#     ).reshape(-1, 1)
# )

In [ ]:
# x = [p[0] for p in thetas]
# y = [p[1] for p in thetas]
# plt.scatter(x, y, s=1.0, alpha=0.5)

In [ ]:
# x = [p[0] for p in vs]
# y = [p[1] for p in vs]
# plt.scatter(x, y, s=1.0, alpha=0.5)

In [ ]:
import matplotlib.pyplot as plt

from spf.rf import reduce_theta_to_positive_y

fig, ax = plt.subplots(3, 2, figsize=(10, 15))

for rx_idx in [0, 1]:  # [0, 1]:
    ax[1, rx_idx].axhline(y=np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
    ax[1, rx_idx].axhline(y=-np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
    trajectory = pf_trajectory_for_phi(rx_idx, ds)
    n = len(trajectory)
    ax[0, rx_idx].scatter(
        range(min(n, ds.mean_phase[f"r{rx_idx}"].shape[0])),
        ds.mean_phase[f"r{rx_idx}"][:n],
        label=f"r{rx_idx} estimated phi",
        s=1.0,
        alpha=1.0,
        color="red",
    )
    ax[0, rx_idx].plot(ds.ground_truth_phis[rx_idx][:n], label="perfect phi")
    ax[1, rx_idx].plot(
        [ds[idx][rx_idx]["ground_truth_theta"] for idx in range(min(n, len(ds)))],
        label=f"r{rx_idx} gt theta",
    )
    reduced_gt_theta = np.array(
        [
            reduce_theta_to_positive_y(ds[idx][rx_idx]["ground_truth_theta"])
            for idx in range(min(n, len(ds)))
        ]
    )
    ax[1, rx_idx].plot(
        reduced_gt_theta,
        label=f"r{rx_idx} reduced gt theta",
    )

    xs = np.array([x["theta"] for x in trajectory])
    # stds = np.sqrt(np.array([x["P_theta"] for x in trajectory]))
    # zscores = (xs - reduced_gt_theta) / stds

    ax[1, rx_idx].plot(xs, label="PF-x", color="orange")
    # ax[1, rx_idx].fill_between(
    #     np.arange(xs.shape[0]),
    #     xs - stds,
    #     xs + stds,
    #     label="EKF-std",
    #     color="orange",
    #     alpha=0.2,
    # )

    ax[0, rx_idx].set_ylabel("radio phi")

    ax[0, rx_idx].legend()
    ax[0, rx_idx].set_title(f"Radio {rx_idx}")
    ax[1, rx_idx].legend()
    ax[1, rx_idx].set_xlabel("time step")
    ax[1, rx_idx].set_ylabel("radio theta")

    # ax[2, rx_idx].hist(zscores.reshape(-1), bins=25)
fig.suptitle("Single ladies (radios) EKF")
fig.savefig(f"{output_prefix}_single_ladies_ekf.png")

In [ ]:
from spf.rf import pi_norm

pi_norm(0.99 * np.pi - (-0.99 * np.pi))

In [ ]:
THIS WORKS

import numpy as np
import scipy

from filterpy.monte_carlo import systematic_resample

# flip the order of the antennas
antenna_spacing = -ds.yaml_config["receivers"][0]["antenna-spacing-m"]
assert antenna_spacing == -ds.yaml_config["receivers"][1]["antenna-spacing-m"]

wavelength = ds.wavelengths[0]
assert wavelength == ds.wavelengths[1]


def resample_from_index(particles, weights, indexes):
    # print(indexes)
    particles[:] = particles[indexes]
    # weights[:] = weights[indexes]
    # add noise to the new samples
    noise = np.random.randn(*particles.shape)
    # noise[:, 0] *= 0.01
    # noise[:, 1] *= 0.01
    noise[:, 0] *= 0.01
    noise[:, 1] *= 0.001
    change_mask = indexes[:-1] == indexes[1:]
    particles[1:][change_mask] += noise[1:][change_mask]
    # weights[1:][change_mask] *= 0.75
    weights.resize(len(particles))
    weights.fill(1.0 / len(weights))


def create_gaussian_particles(mean, std, N):
    particles = np.empty((N, 2))
    particles[:, 0] = mean[0] + (np.random.randn(N) * std[0])
    particles[:, 1] = mean[1] + (np.random.randn(N) * std[1])
    return particles


def predict(particles, std, dt=1.0):
    N = len(particles)
    particles[:, 0] += dt * particles[:, 1]
    # particles[:, 0] += np.random.randn(particles.shape[0]) * 0.0001
    # particles[:, 1] += np.random.randn(particles.shape[0]) * 0.0001


def hx_pf(x):
    return np.sin(x[:, 0]) * antenna_spacing * 2 * np.pi / wavelength


def theta_phi_to_p(theta, phi, full_p):
    theta_bins = full_p.shape[0]
    phi_bins = full_p.shape[1]
    theta_bin = int(theta_bins * (theta + np.pi) / (2 * np.pi)) % theta_bins
    phi_bin = int(phi_bins * (phi + np.pi) / (2 * np.pi)) % phi_bins
    return full_p[theta_bin, phi_bin]


def update(particles, weights, z, R):
    # weights *= scipy.stats.norm(hx_pf(particles), R).pdf(z)
    # print(weights.shape, z.shape, particles.shape)
    # print(z)
    for idx in range(weights.shape[0]):
        weights[idx] *= theta_phi_to_p(particles[idx, 0], z, full_p=full_p)
        #weights[idx] *= scipy.stats.norm(0, 0.0001).pdf(particles[idx, 1])
    weights += 1.0e-300  # avoid round-off to zero
    weights /= sum(weights)  # normalize


def estimate(particles, weights):
    mean = np.average(particles, weights=weights, axis=0)
    var = np.average((particles - mean) ** 2, weights=weights, axis=0)
    return mean, var


def neff(weights):
    return 1.0 / np.sum(np.square(weights))


N = 128 * 8 *8
particles = create_gaussian_particles(mean=np.array([0, 0]), std=(2, 0.01), N=N)
weights = np.ones((N,)) / N
trajectory = []
thetas = []
vs = []
for rx_idx in [0]:
    for idx in range(len(ds)):
        particles[:, 0] = pi_norm(particles[:, 0])
        predict(particles=particles, std=(0.5, 0.1), dt=0.1)
        particles[:, 0] = pi_norm(particles[:, 0])
        z = np.array(ds[idx][rx_idx]["mean_phase_segmentation"])
        update(particles=particles, weights=weights, z=z, R=2)
        particles[:, 0] = pi_norm(particles[:, 0])
        # print(neff(weights))
        # print(particles.shape)
        # resample if too few effective particles
        if neff(weights) < N / 2:
            # print("RESAMPLE")
            indexes = systematic_resample(weights)
            resample_from_index(particles, weights, indexes)
            assert np.allclose(weights, 1 / N)
        mu, var = estimate(particles, weights)
        # particles = create_gaussian_particles(mean=mu, std=np.sqrt(var), N=N)
        trajectory.append({"theta": mu[0]})
        for particle in particles:
            thetas.append((idx, particle[0]))
            vs.append((idx, particle[1]))

In [ ]:
WORKS2
import numpy as np
import scipy

from filterpy.monte_carlo import systematic_resample

# flip the order of the antennas
antenna_spacing = -ds.yaml_config["receivers"][0]["antenna-spacing-m"]
assert antenna_spacing == -ds.yaml_config["receivers"][1]["antenna-spacing-m"]

wavelength = ds.wavelengths[0]
assert wavelength == ds.wavelengths[1]


def resample_from_index(particles, weights, indexes):
    # print(indexes)
    particles[:] = particles[indexes]
    # weights[:] = weights[indexes]
    # add noise to the new samples
    noise = np.random.randn(*particles.shape)
    # noise[:, 0] *= 0.01
    # noise[:, 1] *= 0.01
    noise[:, 0] *= 0.01
    noise[:, 1] *= 0.01
    change_mask = indexes[:-1] == indexes[1:]
    particles[1:][change_mask] += noise[1:][change_mask]
    # weights.resize(len(particles))
    weights.fill(1.0 / len(weights))
    weights[1:][change_mask] *= 0.01
    weights /= sum(weights)  # normalize


def create_gaussian_particles(mean, std, N):
    particles = np.empty((N, 2))
    particles[:, 0] = mean[0] + (np.random.randn(N) * std[0])
    particles[:, 1] = mean[1] + (np.random.randn(N) * std[1])
    return particles


def predict(particles, std, dt=1.0):
    N = len(particles)
    particles[:, 0] += dt * particles[:, 1]
    # particles[:, 0] += np.random.randn(particles.shape[0]) * 0.0001
    # particles[:, 1] += np.random.randn(particles.shape[0]) * 0.0001


def hx_pf(x):
    return np.sin(x[:, 0]) * antenna_spacing * 2 * np.pi / wavelength


def theta_phi_to_p(theta, phi, full_p):
    theta_bins = full_p.shape[0]
    phi_bins = full_p.shape[1]
    theta_bin = int(theta_bins * (theta + np.pi) / (2 * np.pi)) % theta_bins
    phi_bin = int(phi_bins * (phi + np.pi) / (2 * np.pi)) % phi_bins
    return full_p[theta_bin, phi_bin]


def update(particles, weights, z, R):
    # weights *= scipy.stats.norm(hx_pf(particles), R).pdf(z)
    # print(weights.shape, z.shape, particles.shape)
    # print(z)
    for idx in range(weights.shape[0]):
        weights[idx] *= theta_phi_to_p(particles[idx, 0], z, full_p=full_p)
        # weights[idx] *= scipy.stats.norm(0, 0.0001).pdf(particles[idx, 1])
    weights += 1.0e-300  # avoid round-off to zero
    weights /= sum(weights)  # normalize


def estimate(particles, weights):
    mean = np.average(particles, weights=weights, axis=0)
    var = np.average((particles - mean) ** 2, weights=weights, axis=0)
    return mean, var


def neff(weights):
    return 1.0 / np.sum(np.square(weights))


N = 128 * 8
particles = create_gaussian_particles(mean=np.array([0, 0]), std=(2, 0.01), N=N)
weights = np.ones((N,)) / N
trajectory = []
thetas = []
vs = []
for rx_idx in [0]:
    for idx in range(len(ds)):
        particles[:, 0] = pi_norm(particles[:, 0])
        predict(particles=particles, std=(0.5, 0.1), dt=0.1)
        particles[:, 0] = pi_norm(particles[:, 0])
        z = np.array(ds[idx][rx_idx]["mean_phase_segmentation"])
        update(particles=particles, weights=weights, z=z, R=2)
        particles[:, 0] = pi_norm(particles[:, 0])
        # print(neff(weights))
        # print(particles.shape)
        # resample if too few effective particles
        if neff(weights) < N / 2:
            # print("RESAMPLE")
            indexes = systematic_resample(weights)
            resample_from_index(particles, weights, indexes)
            # assert np.allclose(weights, 1 / N)
        mu, var = estimate(particles, weights)
        # particles = create_gaussian_particles(mean=mu, std=np.sqrt(var), N=N)
        trajectory.append({"theta": mu[0]})
        for particle in particles:
            thetas.append((idx, particle[0]))
            vs.append((idx, particle[1]))

In [ ]:
from numpy.random import randn
import math
import sympy
from sympy.abc import alpha, x, y, v, w, R, theta
from sympy import symbols, Matrix

sympy.init_printing(use_latex="mathjax", fontsize="16pt")
time = symbols("t")
d = v * time
beta = (d / w) * sympy.tan(alpha)
r = w / sympy.tan(alpha)

fxu = Matrix(
    [
        [x - r * sympy.sin(theta) + r * sympy.sin(theta + beta)],
        [y + r * sympy.cos(theta) - r * sympy.cos(theta + beta)],
        [theta + beta],
    ]
)
F = fxu.jacobian(Matrix([x, y, theta]))
F

from filterpy.stats import plot_covariance_ellipse
from math import sqrt, tan, cos, sin, atan2
import matplotlib.pyplot as plt

dt = 1.0

from math import atan2
from math import sqrt


def H_of(x, landmark_pos):
    """compute Jacobian of H matrix where h(x) computes
    the range and bearing to a landmark for state x"""

    px = landmark_pos[0]
    py = landmark_pos[1]
    hyp = (px - x[0, 0]) ** 2 + (py - x[1, 0]) ** 2
    dist = sqrt(hyp)

    H = array(
        [
            [-(px - x[0, 0]) / dist, -(py - x[1, 0]) / dist, 0],
            [(py - x[1, 0]) / hyp, -(px - x[0, 0]) / hyp, -1],
        ]
    )
    return H


def Hx(x, landmark_pos):
    """takes a state variable and returns the measurement
    that would correspond to that state.
    """
    px = landmark_pos[0]
    py = landmark_pos[1]
    dist = sqrt((px - x[0, 0]) ** 2 + (py - x[1, 0]) ** 2)

    Hx = array([[dist], [atan2(py - x[1, 0], px - x[0, 0]) - x[2, 0]]])
    return Hx


from filterpy.kalman import ExtendedKalmanFilter as EKF
from numpy import array, sqrt


class RobotEKF(EKF):
    def __init__(self, dt, wheelbase, std_vel, std_steer):
        EKF.__init__(self, 3, 2, 2)
        self.dt = dt
        self.wheelbase = wheelbase
        self.std_vel = std_vel
        self.std_steer = std_steer

        a, x, y, v, w, theta, time = symbols("a, x, y, v, w, theta, t")
        d = v * time
        beta = (d / w) * sympy.tan(a)
        r = w / sympy.tan(a)

        self.fxu = Matrix(
            [
                [x - r * sympy.sin(theta) + r * sympy.sin(theta + beta)],
                [y + r * sympy.cos(theta) - r * sympy.cos(theta + beta)],
                [theta + beta],
            ]
        )

        self.F_j = self.fxu.jacobian(Matrix([x, y, theta]))
        self.V_j = self.fxu.jacobian(Matrix([v, a]))

        # save dictionary and it's variables for later use
        self.subs = {x: 0, y: 0, v: 0, a: 0, time: dt, w: wheelbase, theta: 0}
        (
            self.x_x,
            self.x_y,
        ) = (
            x,
            y,
        )
        self.v, self.a, self.theta = v, a, theta

    def predict(self, u):
        self.x = self.move(self.x, u, self.dt)
        self.subs[self.x_x] = self.x[0, 0]
        self.subs[self.x_y] = self.x[1, 0]

        self.subs[self.theta] = self.x[2, 0]
        self.subs[self.v] = u[0]
        self.subs[self.a] = u[1]

        F = array(self.F_j.evalf(subs=self.subs)).astype(float)
        V = array(self.V_j.evalf(subs=self.subs)).astype(float)

        # covariance of motion noise in control space
        M = array([[self.std_vel**2, 0], [0, self.std_steer**2]])

        self.P = F @ self.P @ F.T + V @ M @ V.T
        print("P", self.P.shape, "F", F.shape)

    def move(self, x, u, dt):
        hdg = x[2, 0]
        vel = u[0]
        steering_angle = u[1]
        dist = vel * dt

        if abs(steering_angle) > 0.001:  # is robot turning?
            beta = (dist / self.wheelbase) * tan(steering_angle)
            r = self.wheelbase / tan(steering_angle)  # radius

            dx = np.array(
                [
                    [-r * sin(hdg) + r * sin(hdg + beta)],
                    [r * cos(hdg) - r * cos(hdg + beta)],
                    [beta],
                ]
            )
        else:  # moving in straight line
            dx = np.array([[dist * cos(hdg)], [dist * sin(hdg)], [0]])
        return x + dx


def residual(a, b):
    """compute residual (a-b) between measurements containing
    [range, bearing]. Bearing is normalized to [-pi, pi)"""
    y = a - b
    y[1] = y[1] % (2 * np.pi)  # force in range [0, 2 pi)
    if y[1] > np.pi:  # move to [-pi, pi)
        y[1] -= 2 * np.pi
    return y


def z_landmark(lmark, sim_pos, std_rng, std_brg):
    x, y = sim_pos[0, 0], sim_pos[1, 0]
    d = np.sqrt((lmark[0] - x) ** 2 + (lmark[1] - y) ** 2)
    a = atan2(lmark[1] - y, lmark[0] - x) - sim_pos[2, 0]
    z = np.array([[d + randn() * std_rng], [a + randn() * std_brg]])
    return z


def ekf_update(ekf, z, landmark):
    print("EKF UPDATE", "Z", z.shape)
    print("\t", "x", ekf.x.shape, "Hx", Hx(ekf.x, landmark).shape)
    print("\t", "x", ekf.x.shape, "H_of", H_of(ekf.x, landmark).shape)
    ekf.update(
        z, HJacobian=H_of, Hx=Hx, residual=residual, args=(landmark), hx_args=(landmark)
    )


def run_localization(
    landmarks,
    std_vel,
    std_steer,
    std_range,
    std_bearing,
    step=10,
    ellipse_step=20,
    ylim=None,
):
    ekf = RobotEKF(dt, wheelbase=0.5, std_vel=std_vel, std_steer=std_steer)
    ekf.x = array([[2, 6, 0.3]]).T  # x, y, steer angle
    ekf.P = np.diag([0.1, 0.1, 0.1])
    ekf.R = np.diag([std_range**2, std_bearing**2])

    sim_pos = ekf.x.copy()  # simulated position
    # steering command (vel, steering angle radians)
    u = array([1.1, 0.01])

    plt.figure()
    plt.scatter(landmarks[:, 0], landmarks[:, 1], marker="s", s=60)

    track = []
    for i in range(2):
        sim_pos = ekf.move(sim_pos, u, dt / 10.0)  # simulate robot
        track.append(sim_pos)

        if i % step == 0:
            print("U", u.shape)
            ekf.predict(u=u)
            print("x", ekf.x.shape)

            if i % ellipse_step == 0:
                plot_covariance_ellipse(
                    (ekf.x[0, 0], ekf.x[1, 0]),
                    ekf.P[0:2, 0:2],
                    std=6,
                    facecolor="k",
                    alpha=0.3,
                )

            x, y = sim_pos[0, 0], sim_pos[1, 0]
            for lmark in landmarks:
                z = z_landmark(lmark, sim_pos, std_range, std_bearing)
                ekf_update(ekf, z, lmark)

            if i % ellipse_step == 0:
                plot_covariance_ellipse(
                    (ekf.x[0, 0], ekf.x[1, 0]),
                    ekf.P[0:2, 0:2],
                    std=6,
                    facecolor="g",
                    alpha=0.8,
                )
    track = np.array(track)
    plt.plot(track[:, 0], track[:, 1], color="k", lw=2)
    plt.axis("equal")
    plt.title("EKF Robot localization")
    if ylim is not None:
        plt.ylim(*ylim)
    plt.show()
    return ekf